# Atividade 03 - Arquivo Base

Descrição da atividade: Clusterizando Dados de Estatísticas Bancárias por Região Geográfica do Brasil

Objetivo: Implementar uma solução de clusterização para comparar cidades na mesma região geográfica 


# Região Sudeste

Atividade: 
Utilize os dados dos **Relatório por município e agência** do ESTBAN [em formato csv](https://www4.bcb.gov.br/fis/cosif/estban.asp?frame=1) de todas as cidades da região selecionada no ano de 2021 para encontrar similaridades no perfil de obtenção de crédito agropecuário e financiamento imobiliário entre cidades. 

Agrupe as colunas `VERBETE_163_FIN_RURAIS_AGRICUL_CUST/INVEST`, `VERBETE_164_FIN_RURAIS_PECUAR_CUST/INVEST`, `VERBETE_165_FIN_RURAIS_AGRICUL_COMERCIALIZ` e `VERBETE_166_FIN_RURAIS_PECUARIA_COMERCIALIZ` somando seus valores e dividindo pela população para calcular o volume total de financiamentos agropecuários em uma nova coluna `fin_agro_per_capita`. Modifique, também, a coluna `VERBETE_169_FINANCIAMENTOS_IMOBILIARIOS` para `fin_imob_per_capita` . Nos dois casos, divida os valores brutos médios anuais após agrupar por cidade pela população estimada total de cada município, para o ano de 2021.
 
1. Utilize o K-Means para encontrar 5 clusters nessas duas variáveis. Explique se você consegue enxergar alguma relação entre as cidades de cada cluster e exiba um mapa cloroplético.
2. Pesquise alguma das variáveis do censo do IBGE (por exemplo: o PIB de cada município) e apresente um histograma ou o box-plot dessa variável para cada cluster. Explique se essa variável tem comportamento diferente em cada cluster.
Usando o critério de inércia, varie a quantidade de clusters entre 2 e 10.
3. Plote o gráfico de cotovelo e identifique qual quantidade de clusters você recomendaria nesse problema.
4. O projeto de lei suplementar 316/09 propôs uma classificação de municípios em cinco categorias:


> - Rural, se tiver população inferior a 50 mil habitantes, valor adicionado da agropecuária superior a uma terça parte do PIB municipal e densidade demográfica inferior a 80 habitantes por quilômetro quadrado;
>
> - Relativamente rural, se tiver população inferior a 50 mil habitantes, valor adicionado da agropecuária entre uma terça parte e quinze centésimos do PIB municipal e densidade demográfica inferior a 80 habitantes por quilômetro quadrado;
>
> - De pequeno porte, se tiver população inferior a 50 mil habitantes, valor adicionado da agropecuária inferior a quinze centésimos do PIB municipal e densidade demográfica inferior a 80 habitantes por quilômetro quadrado, ou se tiver população inferior a 20 mil habitantes e densidade populacional superior a 80 habitantes por quilômetro quadrado;
>
> - De médio porte, se tiver população entre 50 mil e cem mil habitantes, ou se tiver densidade demográfica superior a 80 habitantes por quilômetro quadrado e população entre 20 mil e 50 mil habitantes;
>
> - De grande porte, se tiver população superior a cem mil habitantes.

4. (cont) Recupere os dados necessários para montar esta classificação a partir da API do IBGE. Existe alguma similaridade entre os clusters obtidos e a classificação do municípío? Calcule o percentual de clusters pertencentes a cada classe. Existe alguma relação entre eles? Exiba um mapa cloroplético dos municípios de acordo com essa classe.





In [ ]:
import pandas as pd
import numpy as np

A sintaxe a seguir armazena os dados de **Relatório por município e agência** do ESTBAN em um único dataframe `df`. [Obtenha os arquivos CSV](https://www4.bcb.gov.br/fis/cosif/estban.asp?frame=1) e carrege para o Google Colab todos os dados relativos ao ano de 2021. Não se esqueça de ajustar o intervalo de busca no laço iterativo.

In [ ]:
df = pd.DataFrame()
for i in range(1,13): #Janeiro até Março (01 a 12)
    if i<10:    
        filename = "20210"+str(i)+"_ESTBAN_AG.CSV" # Geração do nome do arquivo
        dfAux = pd.read_csv(filename, sep=";",on_bad_lines='skip',encoding = "ISO-8859-1",  skiprows=2) # Para manipular o carregamento de CSV do ESTBAN
        df = pd.concat([df,dfAux], ignore_index=True) # Concatenação de dataframes
    else:
        filename = "2021"+str(i)+"_ESTBAN_AG.CSV" # Geração do nome do arquivo
        dfAux = pd.read_csv(filename, sep=";",on_bad_lines='skip',encoding = "ISO-8859-1",  skiprows=2) # Para manipular o carregamento de CSV do ESTBAN
        df = pd.concat([df,dfAux], ignore_index=True) # Concatenação de dataframes

In [ ]:
df.tail(10) 

Verifique que algumas linhas possuem CODMUN `NaN`:

In [ ]:
df[df['CODMUN'].isna()]

Para a análise nesta atividade, vamos ignorar os bancos 

In [ ]:
df = df.dropna(subset=['CODMUN'])
df[df['CODMUN'].isna()]

Podemos visualizar quais colunas estão presentes no dataset:

In [ ]:
df.columns

Vamos utilizar as colunas: `VERBETE_163_FIN_RURAIS_AGRICUL_CUST/INVEST`,
       `VERBETE_164_FIN_RURAIS_PECUAR_CUST/INVEST`,
       `VERBETE_165_FIN_RURAIS_AGRICUL_COMERCIALIZ`,
       `VERBETE_166_FIN_RURAIS_PECUARIA_COMERCIALIZ`, e `VERBETE_169_FINANCIAMENTOS_IMOBILIARIOS`, que correspondem às colunas `23-26` e `28` do nosso dataframe:

In [ ]:
cols = df.columns
selectedCols = []
# selecionando colunas 0-6, 20, 28 e última coluna do dataframe
selectedCols = list(cols[0:7]) + list(cols[23:27]) + [cols[28],cols[-1]]
selectedCols

Vamos manter apenas essas colunas no nosso dataframe. Podemos fazer da forma a seguir:

In [ ]:
df = df.drop(df.columns.difference(selectedCols), axis=1)
df.head()

Vamos verificar os `dtypes` do nosso dataframe

In [ ]:
df.dtypes

Verifique que o tipo de dados da coluna `CODMUN_IBGE` está em float. Por ser um código, o ideal é que o tipo esteja como string, sem o decimal. Vamos converter inicialmente para inteiro e, posteriormente, para string:

In [ ]:
df = df.astype({'CODMUN_IBGE': 'int64'})
df = df.astype({'CODMUN_IBGE': 'str'})
df.dtypes

In [ ]:
df.head()

Vamos agrupar os dados a partir do codigo de município IBGE. Iremos selecionar as colunas que contém a informação que queremos agrupar. Vamos selecionar as devidas colunas 

In [ ]:
selectedCols = df.columns[7:-1]
selectedCols

Em seguida, agruparemos pela variável CODMUN_IBGE, calculando a média do saldo de financiamentos agropecuários e imobiliários.

OBS: O dataframe gerado irá conter a média do saldo de financiamento (agropecuários e imobiliários) de todos os bancos em uma cidade. É possível (e provável) que agências tenham sido criadas ou encerradas durante esse tempo. Portanto, é recomendado fazer uma avaliação mais minuciosa da evolução desses dados ao longo do tempo.

In [ ]:
groupedDf = df.groupby(["CODMUN_IBGE"])[selectedCols].agg('mean')
groupedDf.reset_index(inplace = True)
groupedDf.set_index("CODMUN_IBGE", inplace = True)
groupedDf.head()

Vamos inserir no dataframe a informação populacional. Utilizaremos a estimativa populacional para 2021:

In [ ]:
# Obtendo os dados populacionais - Todas as regiões
import requests                    # api module
import json
url = "https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/2021/variaveis/9324?localidades=N6[N2[1,2,3,4,5]]"
response = requests.get(url)
population = response.json()
for item in population:
for key in item['resultados']:
    populationJson = key
populationJson.pop('classificacoes')
dfPopulation = pd.json_normalize(populationJson['series'])
dfPopulation.rename(columns = {dfPopulation.columns[0]:'CODMUN_IBGE', 
                         dfPopulation.columns[-1]: 'POPULACAO'},
             inplace = True)
dfPopulation[['CIDADE','UF']] = dfPopulation[dfPopulation.columns[-2]].str.split(' - ',1).tolist()
dfPopulation.set_index('CODMUN_IBGE', inplace = True)
dfPopulation.drop(dfPopulation.columns[0:3], axis=1,inplace=True)
dfPopulation = dfPopulation.astype({'POPULACAO': 'int64'})
dfPopulation.head()

Vamos obter, também, a área de cada município:

In [ ]:
# Obtendo a área territorial de todos os municípios Brasileiros
url = "https://servicodados.ibge.gov.br/api/v3/agregados/1301/periodos/2010/variaveis/615?localidades=N6[N2[1,2,3,4,5]]"
response = requests.get(url)
area = response.json()
for item in area:
  for key in item['resultados']:
    areaJson = key
areaJson.pop('classificacoes')
dfArea = pd.json_normalize(areaJson['series'])
dfArea.drop(columns=[dfArea.columns[1],
                    dfArea.columns[2],
                    dfArea.columns[3]],
           inplace=True)
dfArea.rename(columns = {dfArea.columns[0]:'CODMUN_IBGE', 
                       dfArea.columns[1]: 'AREA'},
             inplace = True)
dfArea.set_index('CODMUN_IBGE', inplace = True)
dfArea = dfArea.astype({'AREA': 'float'})
dfArea.head()

Vamos agrupar oS dataframe de população e área ao dataframe dos financiamentos. Os municípios que não apresentarem saldo de financiamento (não possuírem agências), teão valores zero atribuídos.

In [ ]:
df = pd.DataFrame()
df = dfPopulation.merge(dfArea,left_index=True, right_index=True, how='outer')
df.head()

Vamos, agora, agrupar os dataframes dos dados bancários aos dados do IBGE:

In [ ]:
df = df.merge(groupedDf,left_index=True, right_index=True, how='outer').fillna(0)
df.head()

In [ ]:
df.dtypes

Vamos somar as colunas de financiamento agropecuário, dividir pela população e salva-lo em uma nova coluna. Também, vamos calcular os financiamentos imobiliários per capita:

In [ ]:
df['fin_agro_per_capita'] = df.iloc[:,4:8].sum(axis=1)/df['POPULACAO']
df['fin_imob_per_capita'] = df.iloc[:,8]/df['POPULACAO']
df.head()

Agora podemos excluir as demais colunas do dataframe:

In [ ]:
selectedCols = ["POPULACAO","CIDADE","UF","AREA","fin_agro_per_capita","fin_imob_per_capita"]
df = df.drop(df.columns.difference(selectedCols), axis=1)
df.head()

In [ ]:
df.describe()

In [ ]:
from sklearn.cluster import KMeans
import plotly.express as px

Iremos selecionar apenas os municípios do estado de Minas Gerais (MG);

In [ ]:
dfMG = df[df['UF']=='MG']  #colocar demais da região sudeste
dfMG.head()

In [ ]:
dfRJ = df[df['UF']=='RJ'] 
dfRJ.head()

In [ ]:
dfSP = df[df['UF']=='SP']  
dfSP.head()

In [ ]:
dfES = df[df['UF']=='ES']  
dfES.head()

In [ ]:
dfMG.describe()

In [ ]:
dfSudeste = pd.concat([dfMG,dfRJ,dfSP,dfES])
dfSudeste.head() # sudeste, fazer a partir desse valor

In [ ]:
X = dfSudeste[['fin_agro_per_capita', 'fin_imob_per_capita']].to_numpy()

In [ ]:
cl = KMeans(n_clusters=5, random_state=0).fit(X)
clusters = pd.DataFrame(cl.labels_, columns=['cluster'])

detail = pd.merge(left=dfSudeste.reset_index(), right=clusters, left_index=True, right_index=True)
detail = detail.astype({'cluster': 'str'})

fig = px.scatter(detail, x="fin_agro_per_capita", y="fin_imob_per_capita", color="cluster",hover_name='CIDADE')

fig.show() #talvez seja interessante utilizar os estados como marcadores.

In [ ]:
url = 'https://servicodados.ibge.gov.br/api/v3/malhas/regioes/3?formato=application/vnd.geo+json&qualidade=minima&intrarregiao=municipio'
response = requests.get(url)
shapesJson = response.json()



In [ ]:
detail['cluster'] = detail['cluster'].astype(str)

brazilMap = px.choropleth_mapbox(detail,
                geojson=shapesJson, 
                locations='CODMUN_IBGE',
                hover_name = 'CIDADE',
                color='cluster', 
                featureidkey="properties.codarea",
                )
brazilMap.update_layout(mapbox_style="open-street-map", 
                            mapbox_zoom=5,
                            title="Clusters - KMeans")
brazilMap.show()

Justificativa:

Região litoral sudeste devido a aréa não tem espaço para desenvolver atividades agropecuarias, com isso predonomina fundos imobiliarios, em regiões mais afastadas e interiores, tem um aumento na produção agropecuaria, como por exemplo: Paranapanema. A região de Minas gerais predomina o agronegocio.

2: Pesquise alguma das variáveis do censo do IBGE (por exemplo: o PIB de cada município) e apresente um histograma ou o box-plot dessa variável para cada cluster. Explique se essa variável tem comportamento diferente em cada cluster. Usando o critério de inércia, varie a quantidade de clusters entre 2 e 10.



In [ ]:
url = 'https://servicodados.ibge.gov.br/api/v3/agregados/5938/periodos/2012/variaveis/37?localidades=N6[N2[3]]'
response = requests.get(url)
pib = response.json()
for item in pib:
    for key in item['resultados']:
        rendJson = key
rendJson.pop('classificacoes')
dfpib = pd.json_normalize(rendJson['series'])
dfpib.head()


In [ ]:
dfpib.rename(columns = {'localidade.nome':'nome'}, inplace = True)
dfpib[['CIDADE','UF']]=dfpib.nome.str.split(' - ',expand=True)
dfpib.rename(columns = {'serie.2012':'pib'}, inplace = True)


In [ ]:
dfpib.drop(columns=['localidade.id','localidade.nivel.id','localidade.nivel.nome','nome','UF'], inplace = True)


In [ ]:
dfpib

In [ ]:
dfSudeste_pib = detail.merge(dfpib, how='inner', on='CIDADE')
dfSudeste_pib

In [ ]:
X = dfSudeste_pib[['pib', 'cluster']].to_numpy()

In [ ]:
#detail2 = detail2.astype({'pib': 'int64'})


In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px 


for z in range(2, 9):#9
    cl = KMeans(n_clusters=z, random_state=0).fit(X) # fazer de 2 a 10
    clusters = pd.DataFrame(cl.labels_, columns=['new_cluster'])

    detail2 = pd.merge(left=dfSudeste_pib.reset_index(), right=clusters, left_index=True, right_index=True)
    detail2 = detail2.astype({'new_cluster': 'str'})

    #fig = px.scatter(detail2, x="pib", y="cluster", color="new_cluster",hover_name='CIDADE')

    #fig.show()
  

    fig = px.box(detail2, x="new_cluster", y="pib") 
  
    fig.show()

In [ ]:
detail2

Justificativa:

De acordo com a ordenação dos cluesteres anteriores por (Fin_agro e Fin_imob) temos a nova execução de um Kmeans ( cluster e pib), assim gerando novos cluestres, ao analizar os novos clusters e o pib, conseguimos notar que existem clusters que tomam a maior parte dos dados e outros que quase não possuem, isso se deve ao fato da predominancia dos clusteres anteriores, em regioes com alto indice imobiliario, dessa forma ao executarmos com:

2 clusteres: Vemos um totalmente predominando os dados enquanto o segundo, com quase nada do pib.

3 clusteres: O mesmo acontece, um predomina e os outros dois quase vazios

4 clusteres: Nesse fica dividido, dois com a maior parte dos dados e 2 quase vazios

5 clusteres: Três com os dados e dois quase sem

6 clusteres: Quatro com os dados e 2 quase sem

7 clusteres: Quatro com os dados e 3 quase sem

8 clusteres: Seis com os dados e 2 quase sem.

Dessa forma é possivel analizar, que quanto maior a quantidade de clusteres maior vai ser a diversificação dos dados, porém ainda existira clusteres que não se adequam.



3 : 
Plote o gráfico de cotovelo e identifique qual quantidade de clusters você recomendaria nesse problema.



In [ ]:
df_elbow = detail2.copy()
df_elbow

In [ ]:
df_elbow.drop(columns=['CIDADE','index','UF'], inplace = True)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X = StandardScaler().fit_transform(df_elbow.values)

# k means determine k
distortions = []
K = range(3, 20, 2)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(X)
    kmeanModel.fit(X)
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()


Justificativa: A curva mostra que o Melhor valor para K, ou seja o numero de clusters é igual a cinco (5)

4 ) O projeto de lei suplementar 316/09 propôs uma classificação de municípios em cinco categorias:
Rural, se tiver população inferior a 50 mil habitantes, valor adicionado da agropecuária superior a uma terça parte do PIB municipal e densidade demográfica inferior a 80 habitantes por quilômetro quadrado;

Relativamente rural, se tiver população inferior a 50 mil habitantes, valor adicionado da agropecuária entre uma terça parte e quinze centésimos do PIB municipal e densidade demográfica inferior a 80 habitantes por quilômetro quadrado;

De pequeno porte, se tiver população inferior a 50 mil habitantes, valor adicionado da agropecuária inferior a quinze centésimos do PIB municipal e densidade demográfica inferior a 80 habitantes por quilômetro quadrado, ou se tiver população inferior a 20 mil habitantes e densidade populacional superior a 80 habitantes por quilômetro quadrado;

De médio porte, se tiver população entre 50 mil e cem mil habitantes, ou se tiver densidade demográfica superior a 80 habitantes por quilômetro quadrado e população entre 20 mil e 50 mil habitantes;

De grande porte, se tiver população superior a cem mil habitantes.

(cont) Recupere os dados necessários para montar esta classificação a partir da API do IBGE. Existe alguma similaridade entre os clusters obtidos e a classificação do municípío? Calcule o percentual de clusters pertencentes a cada classe. Existe alguma relação entre eles? Exiba um mapa cloroplético dos municípios de acordo com essa classe.

In [ ]:
url = 'https://servicodados.ibge.gov.br/api/v3/agregados/1301/periodos/2010/variaveis/616?localidades=N6[N2[3]]'
response = requests.get(url)
densidade = response.json()
for item in densidade:
    for key in item['resultados']:
        rendJson = key
rendJson.pop('classificacoes')
dfdes = pd.json_normalize(rendJson['series'])
dfdes.head()


In [ ]:
dfdes.rename(columns = {'localidade.nome':'nome'}, inplace = True)
dfdes[['CIDADE','UF']]=dfdes.nome.str.split(' - ',expand=True)
dfdes.rename(columns = {'serie.2010':'densidade'}, inplace = True)

In [ ]:
dfdes.drop(columns=['localidade.id','localidade.nivel.id','localidade.nivel.nome','nome','UF'], inplace = True)

In [ ]:
dfdes

In [ ]:
df_t = detail2.merge(dfdes, how='inner', on='CIDADE') # pode ou nao retirar os clusteres
df_t

In [ ]:
df_t.dtypes

In [ ]:
df_t.pib = df_t.pib.astype(int)
df_t.densidade = df_t.densidade.astype(float)


In [ ]:
df_t.POPULACAO.iloc[0]

In [ ]:
df_t["agro"]= pd.NaT

In [ ]:
for x in range(1722):
    df_t.agro.iloc[x]= df_t.fin_agro_per_capita.iloc[x] * df_t.POPULACAO.iloc[x]

In [ ]:
df_t.densidade.iloc[0]

In [ ]:
x = df_t.pib.iloc[0]/3
x

In [ ]:
df_t["tipo"]= pd.NaT

In [ ]:
df_t

In [ ]:
for i in range(1722):
    if df_t.POPULACAO.iloc[i] < 50000 and df_t.agro.iloc[i] > df_t.pib.iloc[i]/3:
        if df_t.densidade.iloc[i] <80:
            df_t["tipo"].iloc[i] = 0
        
    if df_t.POPULACAO.iloc[i] < 50000 and df_t.agro.iloc[i] < df_t.pib.iloc[i]/3 :
        if (df_t.agro.iloc[i] > df_t.pib.iloc[i]*0,15 and df_t.densidade.iloc[i] <80):
            df_t["tipo"].iloc[i] = 1 

        
    if (df_t.POPULACAO.iloc[i] < 50000 and  df_t.agro.iloc[i] < df_t.pib.iloc[i]*0,15):
        if (df_t.densidade.iloc[i] <80):
            df_t["tipo"].iloc[i] = 2
            
    if (df_t.POPULACAO.iloc[i] < 20000 and df_t.densidade.iloc[i] >80):
         df_t["tipo"].iloc[i] = 2



    if (df_t.POPULACAO.iloc[i] >= 50000 and df_t.POPULACAO.iloc[i] <= 100000):
        df_t["tipo"].iloc[i] = 3

    if (df_t.densidade.iloc[i] >80 and df_t.POPULACAO.iloc[i] >= 20000):
        if (df_t.POPULACAO.iloc[i] <= 50000):
            df_t["tipo"].iloc[i] = 3


    if (df_t.POPULACAO.iloc[i] > 100000):
        df_t["tipo"].iloc[i] = 4

In [ ]:
for i in range(1722):
    if df_t.POPULACAO.iloc[i] < 50000 and df_t.agro.iloc[i] > df_t.pib.iloc[i]/3:
        if df_t.densidade.iloc[i] <80:
            df_t["tipo"].iloc[i] = 0
        
    elif df_t.POPULACAO.iloc[i] < 50000 and df_t.agro.iloc[i] < df_t.pib.iloc[i]/3 :
        if (df_t.agro.iloc[i] > df_t.pib.iloc[i]*0,15 and df_t.densidade.iloc[i] <80):
            df_t["tipo"].iloc[i] = 1 

        
    elif (df_t.POPULACAO.iloc[i] < 50000 and  df_t.agro.iloc[i] < df_t.pib.iloc[i]*0,15):
        if (df_t.densidade.iloc[i] <80):
            df_t["tipo"].iloc[i] = 2
            
    elif (df_t.POPULACAO.iloc[i] < 20000 and df_t.densidade.iloc[i] >80):
         df_t["tipo"].iloc[i] = 2



    elif (df_t.POPULACAO.iloc[i] >= 50000 and df_t.POPULACAO.iloc[i] <= 100000):
        df_t["tipo"].iloc[i] = 3

    elif (df_t.densidade.iloc[i] >80 and df_t.POPULACAO.iloc[i] >= 20000):
        if (df_t.POPULACAO.iloc[i] <= 50000):
            df_t["tipo"].iloc[i] = 3


    elif (df_t.POPULACAO.iloc[i] > 100000):
        df_t["tipo"].iloc[i] = 4

In [ ]:
df_t #comparação com clusteres

In [ ]:
df_t.tipo.unique()

In [ ]:
df_t_save = df_t.copy()


In [ ]:
df_t.cluster.unique()

In [ ]:
df_comp = pd.DataFrame(df_t, columns = ['cluster','tipo'])
df_comp

In [ ]:
df_comp = df_comp.astype(int)

In [ ]:
df_mask=df_comp['cluster']==0
filtered_df = df_comp[df_mask]
filtered_df

In [ ]:
freq = filtered_df.groupby(['tipo']).count() 
print(freq)

In [ ]:
df_mask=df_comp['cluster']==1
filtered_df = df_comp[df_mask]
filtered_df

In [ ]:
freq = filtered_df.groupby(['tipo']).count() 
print(freq)

In [ ]:
df_mask=df_comp['cluster']==2
filtered_df = df_comp[df_mask]
filtered_df

In [ ]:
freq = filtered_df.groupby(['tipo']).count() 
print(freq)

In [ ]:
df_mask=df_comp['cluster']==3
filtered_df = df_comp[df_mask]
filtered_df

In [ ]:
freq = filtered_df.groupby(['tipo']).count() 
print(freq)

In [ ]:
df_mask=df_comp['cluster']==4
filtered_df = df_comp[df_mask]
filtered_df

In [ ]:
freq = filtered_df.groupby(['tipo']).count() 
print(freq)

In [ ]:
rural = 0
semirural = 0
pequenoPorte = 0
medioPorte = 0
grandePorte = 0

for x in range(1722):
    if df_comp.tipo.iloc[x]==0:
        rural = rural +1

    if df_comp.tipo.iloc[x]==1:
        semirural = semirural +1

    if df_comp.tipo.iloc[x]==2:
        pequenoPorte = pequenoPorte +1

    if df_comp.tipo.iloc[x]==3:
        medioPorte = medioPorte +1

    if df_comp.tipo.iloc[x]==4:
        grandePorte = grandePorte +1


In [ ]:
print('percentual de rural',rural*100/1722)
print('percentual de semirural',semirural*100/1722)
print('percentual de pequenoPorte',pequenoPorte*100/1722)
print('percentual de medioPorte',medioPorte*100/1722)
print('percentual de grandePorte',grandePorte*100/1722)

print('total',grandePorte*100/1722+ medioPorte*100/1722+pequenoPorte*100/1722+semirural*100/1722+rural*100/1722)


In [ ]:
# AQUI ESTOU MUDANDO O NOME DE TIPO PARA CLUSTER, PQ POR ALGUM MOTIVO SÓ CONSIGO O MAPA ASSIM,KKKK

df_t.rename(columns = {'cluster':'cluster_tes'}, inplace = True)
df_t.rename(columns = {'tipo':'cluster'}, inplace = True)


In [ ]:
df_t

In [ ]:
url = 'https://servicodados.ibge.gov.br/api/v3/malhas/regioes/3?formato=application/vnd.geo+json&qualidade=minima&intrarregiao=municipio'
response = requests.get(url)
shapesJson = response.json()


In [ ]:
df_t['cluster'] = df_t['cluster'].astype(str)

brazilMap = px.choropleth_mapbox(df_t,
                geojson=shapesJson, 
                locations='CODMUN_IBGE',
                hover_name = 'CIDADE',
                color='cluster', 
                featureidkey="properties.codarea",
                )
brazilMap.update_layout(mapbox_style="open-street-map", 
                            mapbox_zoom=5,
                            title="Clusters - KMeans")
brazilMap.show()

justificativa: Os dados monstram que os clusters  (0,4) que possuem maior indice imobiliario representam a maior parte das cidades, sendo o Cluster 4 responsavel por mais da metade desses dados, ao observar em comparação com o tipo de regiãos, vemos a predominância de semi rural e rural seguida por grande porte, isso se dá devido a grande area ocupada por Minas Gerais, que favorece a classificação das zonas, do mesmo modo, podemos entender mediante as demais classificações de frequência, que o acerto de valores de uma classe regional é muito proxima do valor do cluster, como nos clusters 1,2,3 onde predomina um tipo regional e os demais tipos são minimos. 